In [3]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.optimize import curve_fit

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split

from qiskit import pulse  # This is where we access all of our Pulse features!
from qiskit.circuit import Parameter  # This is Parameter Class for variable parameters.
from qiskit.circuit import QuantumCircuit, Gate
from qiskit import schedule

from qiskit.tools.monitor import job_monitor
from numpy import savetxt

import pandas as pd
import scipy.linalg as la

pi = np.pi
qubit = 0

In [4]:
from qiskit import IBMQ
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
backend = provider.get_backend('ibm_oslo')

# sweep range = $[0, \pi/16]$
data
folder: ramsey_data_16_not_centered
job
id:
0: 630c345b65fbdd748201e059
1: 630c34ae19ada766f904bf0a
2: 630c34f6b3145c190b80958e
3: 630c351facf65c9ec1809254
4: 630c354c04b8d5ed019181c1

# sweep range = $[-\pi/64, \pi/64]$
job
id:
0: 630c7d27acf65c28fd80940f
1: 630c7d6fb3145c3979809746
2: 630c7ddb794b2e2e1b347f85
3: 630c7e41acf65cd49380941b
4: 630c7e71acf65c809e80941d

# sweep range = $[-\pi/120, \pi/120]$
job
id:
0: 630c9b481d4bf84119eddc70
1: 630c9b74794b2e32cd347ffa
2: 630c9b99b3145c8be78097d4
3: 630c9bc11d4bf88172eddc77
4: 630c9c091d4bf8ea3ceddc78


# sweep range = $[-\pi/45, \pi/45]$
job
id:
0: 630cdb65794b2e8a9c348135
1: 630cdb8819ada7bcc504c271
2: 630cdba1a7ff837c99784935
3: 630cdbbd19ada72d3404c272
4: 630cdc98ac8baf6eb862c1f4

# sweep range = $[-\pi/45, \pi/45]$
job
id:
0: 630dacaeacf65c9186809a1f
1: 630daccbdce5362b6ca41ec9
2: 630dace4a7ff832b52784d21
3: 630dacffac8baf32e462c62f
4: 630dae671d4bf83a5fede223

# sweep range = $[-\pi/20, \pi/20]$
job id:
0: 630dd73165fbdd9bd201e812
1: 630dd792dce5361e99a41f83
2: 630dd7d304b8d5cdcb9189a3
3: 630dd835ac8baf58da62c6d1
4: 630dd865a7ff83062f784ddb

In [ ]:
scale_factor = 1e-7


def get_job_data(job, average):
    """Retrieve data from a job that has already run.
    Args:
        job (Job): The job whose data you want.
        average (bool): If True, gets the data assuming data is an average.
                        If False, gets the data assuming it is for single shots.
    Return:
        list: List containing job result data.
    """
    job_results = job.result()  # timeout parameter set to 120 s
    result_data = []
    for i in range(len(job_results.results)):
        if average:  # get avg data
            result_data.append(np.real(job_results.get_memory(i)[qubit] * scale_factor))
        else:  # get single data
            result_data.append(job_results.get_memory(i)[:, qubit] * scale_factor)
    return result_data

In [16]:
exp_pos = 1
num_series = 6
job = backend.retrieve_job('630daccbdce5362b6ca41ec9')
print(job.job_id())
raw_ramsey_data = get_job_data(job, average=False)
savetxt('raw_ramsey_exp_data_' + job.job_id() + '_' + str(1 + exp_pos * num_series) + 'to' + str(
    6 + exp_pos * num_series) + '.csv', raw_ramsey_data, delimiter=',')

630daccbdce5362b6ca41ec9


In [5]:
job = backend.retrieve_job('630daccbdce5362b6ca41ec9')

In [7]:
data = pd.read_csv('raw_ramsey_exp_data_' + job.job_id() + '_' + str(1 + exp_pos * num_series) + 'to' + str(
    6 + exp_pos * num_series) + '.csv', sep=",", header=None)
data = data.applymap(lambda s: complex(s.replace('i', 'j'))).values

zero_data = data[0]
one_data = data[1]
two_data = data[2]


def reshape_complex_vec(vec):
    """Take in complex vector vec and return 2d array w/ real, imag entries. This is needed for the learning.
    Args:
        vec (list): complex vector of data
    Returns:
        list: vector w/ entries given by (real(vec], imag(vec))
    """
    length = len(vec)
    vec_reshaped = np.zeros((length, 2))
    for i in range(len(vec)):
        vec_reshaped[i] = [np.real(vec[i]), np.imag(vec[i])]
    return vec_reshaped


# Create IQ vector (split real, imag parts)
zero_data_reshaped = reshape_complex_vec(zero_data)
one_data_reshaped = reshape_complex_vec(one_data)
two_data_reshaped = reshape_complex_vec(two_data)

IQ_012_data = np.concatenate((zero_data_reshaped, one_data_reshaped, two_data_reshaped))
print(IQ_012_data.shape)  # verify IQ data shape

NUM_SHOTS = 20000

# construct vector w/ 0's, 1's and 2's (for testing)
state_012 = np.zeros(NUM_SHOTS)  # shots gives number of experiments
state_012 = np.concatenate((state_012, np.ones(NUM_SHOTS)))
state_012 = np.concatenate((state_012, 2 * np.ones(NUM_SHOTS)))
print(len(state_012))

# Shuffle and split data into training and test sets
IQ_012_train, IQ_012_test, state_012_train, state_012_test = train_test_split(IQ_012_data, state_012, test_size=0.5)

# Set up the LDA
LDA_012 = LinearDiscriminantAnalysis()
LDA_012.fit(IQ_012_train, state_012_train)


def count(job, discriminator):
    data = get_job_data(job, average=False)
    sched_data = []
    for i in range(len(data)):
        sched_data.append(reshape_complex_vec(data[i]))
    discrim_data = []
    for j in range(len(sched_data)):
        discrim_data.append(discriminator.predict(sched_data[j]))
    final_result = []
    for k in range(len(discrim_data)):
        result = {'0': 0, '1': 0, '2': 0}
        for l in range(len(discrim_data[k])):
            if discrim_data[k][l] == 0.0:
                result['0'] += 1
            elif discrim_data[k][l] == 1.0:
                result['1'] += 1
            elif discrim_data[k][l] == 2.0:
                result['2'] += 1
            else:
                print('Unexpected behavior')
        final_result.append(result)
    return final_result

FileNotFoundError: [Errno 2] No such file or directory: 'raw_ramsey_exp_data_630daccbdce5362b6ca41ec9_7to12.csv'

In [ ]:
discr_data = count(job, LDA_012)

In [ ]:
np.shape(discr_data)

In [ ]:
ramsey_exp_data = [[discr_data[i]['0'] / 20000, discr_data[i]['1'] / 20000, discr_data[i]['2'] / 20000] for i in
                   range(np.shape(discr_data)[0])]
ramsey_exp_data

In [1]:
filter_data = np.transpose(ramsey_exp_data[0:3])
data = np.transpose(np.dot(la.inv(filter_data), np.transpose(ramsey_exp_data)))

NameError: name 'ramsey_exp_data' is not defined

In [ ]:
savetxt('ramsey_exp_data_' + job.job_id() + "_" + str(1 + exp_pos*num_series) + 'to' + str(6 + exp_pos*num_series) + '.csv', data, delimiter=',')